In [1]:
import pystac_client
from tqdm import tqdm
import pathlib
from datetime import datetime
import requests


In [2]:
STAC_API_URL = 'https://data.inpe.br/bdc/stac/v1/'
BASE_DIR = '../data/goes/goes16'
CHANNELS = ['B01', 'B02', 'B03', 'B13']
START_DATE = datetime.strptime('2025-01-01 13:00', '%Y-%m-%d %H:%M')
END_DATE = datetime.strptime('2025-01-01 13:10', '%Y-%m-%d %H:%M')
# date at format "2025-01-01T13:00:00Z/2025-01-01T14:00:00Z"
DATE_RANGE = f"{START_DATE.strftime('%Y-%m-%dT%H:%M:%SZ')}/{END_DATE.strftime('%Y-%m-%dT%H:%M:%SZ')}"

In [3]:
service = pystac_client.Client.open(STAC_API_URL)
service

<Client id=INPE>

In [4]:
collections = service.get_all_collections()
print(f"First 10 Collections available at {STAC_API_URL}:")
for collection in list(collections)[:10]:
	print(f"- {collection.id}")

First 10 Collections available at https://data.inpe.br/bdc/stac/v1/:
- AMZ1-WFI-L4-SR-1
- mosaic-cbers4a-paraiba-3m-1
- LCC_L8_30_16D_STK_Cerrado-1
- mosaic-landsat-sp-6m-1
- CB2B-CCD-L2-DN-1
- GOES19-L2-CMI-1
- mosaic-s2-paraiba-3m-1
- myd13q1-6.1
- LCC_L8_30_16D_STK_MataAtlantica-1
- mosaic-s2-yanomami_territory-6m-1


In [5]:
collection = service.get_collection("GOES16-L2-CMI-1")
collection

<CollectionClient id=GOES16-L2-CMI-1>

In [6]:
items_search = service.search(
    collections=["GOES16-L2-CMI-1"],
	datetime=DATE_RANGE,
)
print(f"Found {items_search.matched()} items in collection 'GOES16-L2-CMI-1' for datetime range {DATE_RANGE}")

Found 2 items in collection 'GOES16-L2-CMI-1' for datetime range 2025-01-01T13:00:00Z/2025-01-01T13:10:00Z


In [7]:
def download_file(url: str, dest_path: pathlib.Path) -> pathlib.Path:
	filename = url.split('/')[-1]
	dest_file = dest_path / filename
	if dest_file.exists():
		print(f"File already exists: {dest_file}")
		return dest_file
	try:
		response = requests.get(url, stream=True)
		response.raise_for_status()

		size_in_bytes = int(response.headers.get('content-length', 0))
		block_size = 1024  # 1 KB
		progress_bar = tqdm(total=size_in_bytes, unit='iB', unit_scale=True)
		with open(dest_file, 'wb') as file:
			for data in response.iter_content(block_size):
				progress_bar.update(len(data))
				file.write(data)
		progress_bar.close()
		if size_in_bytes != 0 and progress_bar.n != size_in_bytes:
			print("ERROR, something went wrong")
		else:
			print(f"Downloaded: {dest_file}")
	except requests.exceptions.RequestException as e:
		print(f"Failed to download {url}. Error: {e}")

	return dest_file

In [8]:
items = list(items_search.get_items())
for item in items:
	print(f"Item ID: {item.id}")
	print(f"  - Datetime: {item.datetime}")
	print(f"  - Number of assets: {len(item.assets)}")
	for channel in CHANNELS:
		if channel in item.assets:
			asset = item.assets[channel]
			url = asset.href
			print(f"  - Downloading asset '{channel}' from URL: {url}")
			
			# Create directory structure based on date
			date_dir = item.datetime.strftime('%Y/%m/%d')
			product = pathlib.Path('ABI')
			channel = pathlib.Path(channel)
			dest_path = pathlib.Path(BASE_DIR) / product / channel / date_dir
			dest_path.mkdir(parents=True, exist_ok=True)
			
			# Download the file
			_ = download_file(url, dest_path)
		else:
			print(f"  - Asset '{channel}' not found in this item.")

/home/adriano/Doutorado/cap408-goes-lecture/.venv/lib/python3.12/site-packages/pystac_client/item_search.py:925: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


Item ID: GOES16_L2_ABI_202501011310
  - Datetime: 2025-01-01 13:10:00+00:00
  - Number of assets: 18
  - Downloading asset 'B01' from URL: https://data.inpe.br/bdc/data/GOES16-L2-CMI/ch01/2025/01/S10635300_202501011310.nc


100%|██████████| 76.6M/76.6M [00:13<00:00, 5.61MiB/s]


Downloaded: ../data/goes/goes16/ABI/B01/2025/01/01/S10635300_202501011310.nc
  - Downloading asset 'B02' from URL: https://data.inpe.br/bdc/data/GOES16-L2-CMI/ch02_500m/2025/01/S10635316_202501011310.nc


100%|██████████| 427M/427M [00:38<00:00, 11.0MiB/s] 


...

Downloaded: ../data/goes/goes16/ABI/B02/2025/01/01/S10635316_202501011300.nc
  - Downloading asset 'B03' from URL: https://data.inpe.br/bdc/data/GOES16-L2-CMI/ch03/2025/01/S10635302_202501011300.nc


100%|██████████| 83.1M/83.1M [00:07<00:00, 11.2MiB/s]


Downloaded: ../data/goes/goes16/ABI/B03/2025/01/01/S10635302_202501011300.nc
  - Downloading asset 'B13' from URL: https://data.inpe.br/bdc/data/GOES16-L2-CMI/ch13/2025/01/S10635312_202501011300.nc


100%|██████████| 29.5M/29.5M [00:02<00:00, 10.7MiB/s]

Downloaded: ../data/goes/goes16/ABI/B13/2025/01/01/S10635312_202501011300.nc
